In [ ]:
# Pulls GAIA/KIC/GSC data on stellar properties and adds to variability labels

In [1]:
import warnings 
warnings.filterwarnings("ignore")
from astroquery.vizier import Vizier
from astropy.coordinates import Angle
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
kic_cols = ['Teff', 'logg', 'gmag', 'rmag', 'imag', 'zmag', 'Jmag', 'Kmag', '__Fe_H_', 'E_B-V_', 'Av', 'R_']
gaia_cols = ['RA_ICRS', 'DE_ICRS', 'Plx', 'pmRA', 'pmDE']
gsc_cols = ['RA_ICRS', 'DE_ICRS', 'plx', 'pmRA', 'pmDE']

df = pd.read_csv("../data/lc_data_new.out")

df = df[['KIC']]

df[kic_cols] = None
df[gaia_cols] = None

In [3]:
kic = 'V/133/kic'
gaia = 'I/355/gaiadr3'
gsc = 'I/353/gsc242'

v = Vizier(catalog=[kic, gaia, gsc], columns=["**", "+_r"])

In [8]:
results = v.query_object('KIC 757099')
results[0].info()

<Table length=50>
  name   dtype       unit      format                                                          description                                                          n_bad
------- ------- ------------- -------- ---------------------------------------------------------------------------------------------------------------------------- -----
     _r float64                {:6.4f}                                                                       Distance from center (291.043042+36.593779)[FK5/J2000]     0
    KIC   int32                                                                                                        [1,13161029] KIC identifier (running number)     0
RAJ2000 float64           deg {:10.6f}                                                                                                      Right Ascension (J2000)     0
DEJ2000 float64           deg {:10.6f}                                                                                              

In [ ]:
def upd_table(table, cols, id):
    for col in cols:
        idx = df[df['KIC'] == id].index[0]
        df.at[idx, col] = table[col]

for id in tqdm(df['KIC']):
    search = "KIC " + str(id)
    results = v.query_object(search)
    for catalogue in results.keys():
        table = results[catalogue][0]
        if catalogue == kic:
            upd_table(table, kic_cols, id)
        elif catalogue == gaia:
            upd_table(table, gaia_cols, id)
        elif gaia not in results.keys() and catalogue == gsc:
            upd_table(table, gsc_cols, id)     

In [ ]:
df = df.replace(to_replace=-1.0, value=np.nan)
df = df.replace(to_replace=-999, value=np.nan)
df = df.replace(to_replace='NaN', value=np.nan)
df = df.replace(to_replace='--', value=np.nan)
df = df.replace(to_replace=np.inf, value=np.nan)
df = df.replace(to_replace=-np.inf, value=np.nan)
df = df.dropna()
df.to_csv("../output/rf_inp.csv", index=False)